In [1]:
!pip install pymongo
from pymongo import MongoClient

In [2]:
host="rhea.isegi.unl.pt"
port="28011"
user="GROUP_11"
password="MTM0MzQ5MzI3NjY4ODM1MzQ2ODc2OTMzMjgyNjQ3NjYyNDcyNjUw"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

In [3]:
db = client.contracts
eu = db.eu_2

In [4]:
db.iso_codes.find_one()

{'_id': ObjectId('5e611db7e39120912368d89a'),
 'name': 'Åland Islands',
 'alpha-2': 'AX',
 'alpha-3': 'ALA',
 'country-code': '248',
 'iso_3166-2': 'ISO 3166-2:AX',
 'region': 'Europe',
 'sub-region': 'Northern Europe',
 'intermediate-region': '',
 'region-code': '150',
 'sub-region-code': '154',
 'intermediate-region-code': ''}

In [7]:
list(db.iso_codes.find({'alpha-2':'GB'}))

[{'_id': ObjectId('5e611db7e39120912368d926'),
  'name': 'United Kingdom of Great Britain and Northern Ireland',
  'alpha-2': 'GB',
  'alpha-3': 'GBR',
  'country-code': '826',
  'iso_3166-2': 'ISO 3166-2:GB',
  'region': 'Europe',
  'sub-region': 'Northern Europe',
  'intermediate-region': '',
  'region-code': '150',
  'sub-region-code': '154',
  'intermediate-region-code': ''}]

In [11]:
db.iso_codes.update_one({'alpha-2':'GB'},{'$set':{'alpha-2':'UK','name':'United Kingdom'}})

In [12]:
list(db.iso_codes.find({'alpha-2':'GB'}))

[]

In [13]:
list(db.iso_codes.find({'alpha-2':'UK'}))

[{'_id': ObjectId('5e611db7e39120912368d926'),
  'name': 'United Kingdom',
  'alpha-2': 'UK',
  'alpha-3': 'GBR',
  'country-code': '826',
  'iso_3166-2': 'ISO 3166-2:GB',
  'region': 'Europe',
  'sub-region': 'Northern Europe',
  'intermediate-region': '',
  'region-code': '150',
  'sub-region-code': '154',
  'intermediate-region-code': ''}]

In [5]:
bot_year = 2008
top_year = 2020
country_list = ['NO', 'HR', 'HU', 'CH', 'CZ', 'RO', 'LV', 'GR', 'UK', 'SI', 'LT', 'ES', 'FR', 'IE', 'SE', 'NL',
                'PT', 'PL', 'DK', 'MK', 'DE', 'IT', 'BG', 'CY', 'AT', 'LU', 'BE', 'FI', 'EE', 'SK', 'MT', 'LI',
                'IS']

In [58]:
for index in eu.list_indexes():
    print(index['name'])

_id_
year_country_index


In [62]:
eu.create_index([('YEAR',1),('ISO_COUNTRY_CODE',1)],name='year_country_index')

'year_country_index'

In [64]:
eu.drop_index('year_country_index')

1.1

In [18]:
#%%timeit

match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'CPV': {'$exists': True}
    }
}

match_value_euro = {
    '$match':{
        'VALUE_EURO': {'$exists': True}
    }
}

group_cpv_euro_avg = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'AVERAGE_VALUE':{'$avg':'$VALUE_EURO'}
    }
}

group_cpv_euro_avg_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_CPV_VALUE':{'$avg':'$AVERAGE_VALUE'}
    }
}

limit = {
    '$limit':5
}

pipeline_cpv_euro_avg = [match, match_value_euro, group_cpv_euro_avg, group_cpv_euro_avg_2]

agg_cpv_euro_avg = list(eu.aggregate(pipeline_cpv_euro_avg))

agg_cpv_euro_avg[0]['AVERAGE_CPV_VALUE']

2907470.923725584

1.2

In [19]:
group_cpv_count = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'COUNT':{'$sum':1}
    }
}

group_cpv_count_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_COUNT':{'$avg':'$COUNT'}
    }
}

pipeline_cpv_count = [match, group_cpv_count, group_cpv_count_2]

agg_cpv_count = list(eu.aggregate(pipeline_cpv_count))

agg_cpv_count[0]['AVERAGE_COUNT']

66110.65217391304

1.3

In [20]:
match_number_offers = {
    '$match':{
        'NUMBER_OFFERS': {'$exists': True}
    }
}

group_cpv_offer_avg = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'AVERAGE_OFFERS':{'$avg':'$NUMBER_OFFERS'}
    }
}

group_cpv_offer_avg_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_CPV_OFFERS':{'$avg':'$AVERAGE_OFFERS'}
    }
}

pipeline_cpv_offer_avg = [match, match_number_offers, group_cpv_offer_avg, group_cpv_offer_avg_2]

agg_cpv_offer_avg = list(eu.aggregate(pipeline_cpv_offer_avg))

agg_cpv_offer_avg[0]['AVERAGE_CPV_OFFERS']

8.015262068884264

1.4

In [21]:
match_cpv_euro_avg_y_eu = {
    '$match':{
        'B_EU_FUNDS':'Y'
    }
}

pipeline_cpv_euro_avg_y_eu = [match, match_value_euro, match_cpv_euro_avg_y_eu, group_cpv_euro_avg, group_cpv_euro_avg_2]

agg_cpv_euro_avg_y_eu = list(eu.aggregate(pipeline_cpv_euro_avg_y_eu))

agg_cpv_euro_avg_y_eu[0]['AVERAGE_CPV_VALUE']

2792486.692669272

1.5

In [22]:
match_cpv_euro_avg_n_eu = {
    '$match':{
        'B_EU_FUNDS':'N'
    }
}

pipeline_cpv_euro_avg_n_eu = [match, match_value_euro, match_cpv_euro_avg_n_eu, group_cpv_euro_avg, group_cpv_euro_avg_2]

agg_cpv_euro_avg_n_eu = list(eu.aggregate(pipeline_cpv_euro_avg_n_eu))

agg_cpv_euro_avg_n_eu[0]['AVERAGE_CPV_VALUE']

2764243.7013731776

2

In [23]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'CPV': {'$exists': True}
    }
}

group_cpv_count = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'COUNT':{'$sum':1}
    }
}

lookup = {
    '$lookup':{
        'from':'cpv',
        'localField':'_id',
        'foreignField':'cpv_division',
        'as':'CPV'
    }
}

project = {
    '$project':{
        '_id':False,
        'cpv':{ "$arrayElemAt": [ "$CPV", 0] },
        'count':'$COUNT'
    }
}

project_2 = {
    '$project':{
        'cpv':'$cpv.cpv_division_description',
        'count':True
    }
}

pipeline = [match, group_cpv_count, lookup, project, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'count': 230813, 'cpv': 'Construction work'},
 {'count': 93330,
  'cpv': 'Agricultural, forestry, horticultural, aquacultural and apicultural services'},
 {'count': 74564, 'cpv': 'Education and training services'},
 {'count': 1999, 'cpv': 'Services related to the oil and gas industry'},
 {'count': 116795,
  'cpv': 'Sewage, refuse, cleaning and environmental services'},
 {'count': 1161430,
  'cpv': 'Medical equipments, pharmaceuticals and personal care products'},
 {'count': 57040,
  'cpv': 'Petroleum products, fuel, electricity and other sources of energy'},
 {'count': 7703, 'cpv': 'Public utilities'},
 {'count': 18114,
  'cpv': 'Clothing, footwear, luggage articles and accessories'},
 {'count': 6322,
  'cpv': 'Administration, defence and social security services'},
 {'count': 19832,
  'cpv': 'Radio, television, communication, telecommunication and related equipment'},
 {'count': 94466,
  'cpv': 'Business services: law, marketing, consulting, recruitment, printing and security'},
 {'

3

In [25]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'CPV': {'$exists': True},
        'VALUE_EURO': {'$exists': True}
    }
}

group_cpv_euro_avg = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'AVERAGE_VALUE':{'$avg':'$VALUE_EURO'}
    }
}

sort = {
    '$sort':{
        'AVERAGE_VALUE':-1
    }
}

limit = {
    '$limit':5
}

lookup = {
    '$lookup':{
        'from':'cpv',
        'localField':'_id',
        'foreignField':'cpv_division',
        'as':'CPV'
    }
}

project = {
    '$project':{
        '_id':False,
        'cpv':{ "$arrayElemAt": [ "$CPV", 0] },
        'avg':'$AVERAGE_VALUE'
    }
}

project_2 = {
    '$project':{
        'cpv':'$cpv.cpv_division_description',
        'avg':True
    }
}

pipeline = [match, group_cpv_euro_avg, sort, limit, lookup, project, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'avg': 6031764.443306357, 'cpv': 'Construction work'},
 {'avg': 6024557.646283141,
  'cpv': 'Services related to the oil and gas industry'},
 {'avg': 5777134.932820617,
  'cpv': 'Transport services (excl. Waste transport)'},
 {'avg': 4541651.970869796,
  'cpv': 'Hotel, restaurant and retail trade services'},
 {'avg': 4459757.657965389,
  'cpv': 'Architectural, construction, engineering and inspection services'}]

4

In [26]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'CPV': {'$exists': True},
        'VALUE_EURO': {'$exists': True}
    }
}

group_cpv_euro_avg = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'AVERAGE_VALUE':{'$avg':'$VALUE_EURO'}
    }
}

sort = {
    '$sort':{
        'AVERAGE_VALUE':1
    }
}

limit = {
    '$limit':5
}

lookup = {
    '$lookup':{
        'from':'cpv',
        'localField':'_id',
        'foreignField':'cpv_division',
        'as':'CPV'
    }
}

project = {
    '$project':{
        '_id':False,
        'cpv':{ "$arrayElemAt": [ "$CPV", 0] },
        'avg':'$AVERAGE_VALUE'
    }
}

project_2 = {
    '$project':{
        'cpv':'$cpv.cpv_division_description',
        'avg':True
    }
}

pipeline = [match, group_cpv_euro_avg, sort, limit, lookup, project, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'avg': 1261617.4059277952,
  'cpv': 'Agricultural, farming, fishing, forestry and related products'},
 {'avg': 1384645.0949917191,
  'cpv': 'Clothing, footwear, luggage articles and accessories'},
 {'avg': 1461057.6466721345,
  'cpv': 'Laboratory, optical and precision equipments (excl. glasses)'},
 {'avg': 1498052.4052218625,
  'cpv': 'Musical instruments, sport goods, games, toys, handicraft, art materials and accessories'},
 {'avg': 1527093.0988754546,
  'cpv': 'Recreational, cultural and sporting services'}]

5

In [27]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'B_EU_FUNDS':'Y',
        'CPV': {'$exists': True},
        'VALUE_EURO': {'$exists': True}
    }
}

group_cpv_euro_avg = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'AVERAGE_VALUE':{'$avg':'$VALUE_EURO'}
    }
}

sort = {
    '$sort':{
        'AVERAGE_VALUE':-1
    }
}

limit = {
    '$limit':5
}

lookup = {
    '$lookup':{
        'from':'cpv',
        'localField':'_id',
        'foreignField':'cpv_division',
        'as':'CPV'
    }
}

project = {
    '$project':{
        '_id':False,
        'cpv':{ "$arrayElemAt": [ "$CPV", 0] },
        'avg':'$AVERAGE_VALUE'
    }
}

project_2 = {
    '$project':{
        'cpv':'$cpv.cpv_division_description',
        'avg':True
    }
}

pipeline = [match, group_cpv_euro_avg, sort, limit, lookup, project, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'avg': 16257910.058471283,
  'cpv': 'Food, beverages, tobacco and related products'},
 {'avg': 10419752.572405463, 'cpv': 'Repair and maintenance services'},
 {'avg': 8774581.195029901,
  'cpv': 'Transport services (excl. Waste transport)'},
 {'avg': 7059021.661101975, 'cpv': 'Chemical products'},
 {'avg': 6433034.064154094,
  'cpv': 'Architectural, construction, engineering and inspection services'}]

7

In [28]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'CPV': {'$exists': True},
        'VALUE_EURO': {'$exists': True}
    }
}

group_cpv_country_euro_avg = {
    '$group':{
        '_id':{'CPV_DIVISION':'$CPV_DIVISION','COUNTRY':'$ISO_COUNTRY_CODE'},
        'AVERAGE_VALUE':{'$avg':'$VALUE_EURO'}
    }
}

sort = {
    '$sort':{
        'AVERAGE_VALUE':-1
    }
}

group_2 = {
    '$group':{
        '_id':'$_id.COUNTRY',
        'VALUE':{'$first':'$AVERAGE_VALUE'},
        'CPV':{'$first':'$_id.CPV_DIVISION'}
    }
}

lookup_country = {
    '$lookup':{
        'from':'iso_codes',
        'localField':'_id',
        'foreignField':'alpha-2',
        'as':'COUNTRY'
    }
}

lookup_cpv = {
    '$lookup':{
        'from':'cpv',
        'localField':'CPV',
        'foreignField':'cpv_division',
        'as':'CPV'
    }
}

project = {
    '$project':{
        '_id':False,
        'country':{ "$arrayElemAt": [ "$COUNTRY", 0] },
        'cpv':{ "$arrayElemAt": [ "$CPV", 0] },
        'avg':'$VALUE'
    }
}

project_2 = {
    '$project':{
        'country':'$country.name',
        'cpv':'$cpv.cpv_division_description',
        'avg':True
    }
}

pipeline = [match, group_cpv_country_euro_avg, sort, group_2, lookup_country, lookup_cpv, project, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'avg': 8825512.5, 'country': 'Cyprus', 'cpv': 'Public utilities'},
 {'avg': 21520547.6,
  'country': 'Finland',
  'cpv': 'Services related to the oil and gas industry'},
 {'avg': 15488975.092580644,
  'country': 'Switzerland',
  'cpv': 'Radio, television, communication, telecommunication and related equipment'},
 {'avg': 36544684.81752941,
  'country': 'Denmark',
  'cpv': 'Hotel, restaurant and retail trade services'},
 {'avg': 27372424.820588768,
  'country': 'Latvia',
  'cpv': 'Transport services (excl. Waste transport)'},
 {'avg': 12237475.67, 'country': 'Bulgaria'},
 {'avg': 37287103.236710235,
  'country': 'Greece',
  'cpv': 'Education and training services'},
 {'avg': 2193303.2319002375,
  'country': 'North Macedonia',
  'cpv': 'Construction work'},
 {'avg': 39424917.03162162, 'country': 'Ireland', 'cpv': 'Chemical products'},
 {'avg': 23192041.33883759,
  'country': 'Lithuania',
  'cpv': 'Architectural, construction, engineering and inspection services'},
 {'avg': 13830240.232

In [29]:
cpv = '50'
limit = {
    '$limit':5
}

8

In [30]:
match = {
    '$match': {
            '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
            'ISO_COUNTRY_CODE': {'$in': country_list},
            'CPV_DIVISION': {'$eq': cpv},
            'VALUE_EURO': {'$exists': True}
        }
}

bucket = {
    '$bucket':{
        'groupBy':'$VALUE_EURO',
        'boundaries':[0,100000,200000,300000,400000,500000,600000,700000,800000,900000,1000000],
        'default':'>1000000',
        'output':{
            'count':{'$sum':1}
        }
    }
}

project_2 = {
    '$project':{
        '_id':False,
        'bucket':'$_id',
        'count':True
    }
}

pipeline = [match, bucket, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'count': 16658, 'bucket': 0},
 {'count': 11690, 'bucket': 100000},
 {'count': 9105, 'bucket': 200000},
 {'count': 6471, 'bucket': 300000},
 {'count': 4597, 'bucket': 400000},
 {'count': 3989, 'bucket': 500000},
 {'count': 3172, 'bucket': 600000},
 {'count': 2128, 'bucket': 700000},
 {'count': 1992, 'bucket': 800000},
 {'count': 1597, 'bucket': 900000},
 {'count': 29496, 'bucket': '>1000000'}]

9

In [16]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'CPV': {'$ne': None},
        'VALUE_EURO': {'$exists': True},
        'AWARD_VALUE_EURO': {'$exists': True},
        'DT_DISPATCH': {'$exists': True},
        'DT_AWARD': {'$exists': True}
    }
}

project = {
    '$project':{
        '_id':False,
        'CPV_DIVISION':True,
        'TIME_DIFFERENCE':{'$subtract':['$DT_DISPATCH','$DT_AWARD']},
        'VALUE_DIFFERENCE':{'$subtract':['$AWARD_VALUE_EURO','$VALUE_EURO']}
    }
}

group_cpv_difference = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'AVG_TIME_DIFFERENCE':{'$avg':'$TIME_DIFFERENCE'},
        'AVG_VALUE_DIFFERENCE':{'$avg':'$VALUE_DIFFERENCE'}
    }
}

sort = {
    '$sort':{
        'AVG_TIME_DIFFERENCE':-1
    }
}

limit = {
    '$limit':5
}

lookup = {
    '$lookup':{
        'from':'cpv',
        'localField':'_id',
        'foreignField':'cpv_division',
        'as':'CPV'
    }
}

project_2 = {
    '$project':{
        '_id':False,
        'cpv':{ "$arrayElemAt": [ "$CPV", 0] },
        'time_difference':'$AVG_TIME_DIFFERENCE',
        'value_difference':'$AVG_VALUE_DIFFERENCE'
    }
}

project_3 = {
    '$project':{
        'cpv':'$cpv.cpv_division_description',
        'time_difference':True,
        'value_difference':True
    }
}

pipeline = [match, project, group_cpv_difference, sort, limit, lookup, project_2, project_3]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'time_difference': 7715963212.7529125,
  'value_difference': -975537.2121059563,
  'cpv': 'Agricultural, farming, fishing, forestry and related products'},
 {'time_difference': 7301803443.26626,
  'value_difference': -1485746.403433414,
  'cpv': 'Health and social work services'},
 {'time_difference': 6827731228.416379,
  'value_difference': -1838453.5639014537,
  'cpv': 'Public utilities'},
 {'time_difference': 6418660443.587271,
  'value_difference': -959285.731239647,
  'cpv': 'Education and training services'},
 {'time_difference': 6391338770.541692,
  'value_difference': -2661017.8293143203,
  'cpv': 'Services related to the oil and gas industry'}]

10.1

In [33]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list}
    }
}

match_value_euro = {
    '$match':{
        'VALUE_EURO': {'$exists': True}
    }
}

group_country_euro_avg = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'AVERAGE_VALUE':{'$avg':'$VALUE_EURO'}
    }
}

group_country_euro_avg_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_COUNTRY_VALUE':{'$avg':'$AVERAGE_VALUE'}
    }
}

pipeline_country_euro_avg = [match, match_value_euro, group_country_euro_avg, group_country_euro_avg_2]

agg_country_euro_avg = list(eu.aggregate(pipeline_country_euro_avg))

agg_country_euro_avg[0]['AVERAGE_COUNTRY_VALUE']

3419228.553069342

10.2

In [34]:
group_country_count = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'COUNT':{'$sum':1}
    }
}

group_country_count_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_COUNT':{'$avg':'$COUNT'}
    }
}

pipeline_country_count = [match, group_country_count, group_country_count_2]

agg_country_count = list(eu.aggregate(pipeline_country_count))

agg_country_count[0]['AVERAGE_COUNT']

92154.24242424243

10.3

In [35]:
match_number_offers = {
    '$match':{
        'NUMBER_OFFERS': {'$exists': True}
    }
}

group_country_offer_avg = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'AVERAGE_OFFERS':{'$avg':'$NUMBER_OFFERS'}
    }
}

group_country_offer_avg_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_COUNTRY_OFFERS':{'$avg':'$AVERAGE_OFFERS'}
    }
}

pipeline_country_offer_avg = [match, match_number_offers, group_country_offer_avg, group_country_offer_avg_2]

agg_country_offer_avg = list(eu.aggregate(pipeline_country_offer_avg))

agg_country_offer_avg[0]['AVERAGE_COUNTRY_OFFERS']

12.96010834919913

10.4

In [36]:
match_country_euro_avg_y_eu = {
    '$match':{
        'B_EU_FUNDS':'Y'
    }
}

pipeline_country_euro_avg_y_eu = [match, match_value_euro, match_country_euro_avg_y_eu, group_country_euro_avg, group_country_euro_avg_2]

agg_country_euro_avg_y_eu = list(eu.aggregate(pipeline_country_euro_avg_y_eu))

agg_country_euro_avg_y_eu[0]['AVERAGE_COUNTRY_VALUE']

3328575.961350672

10.5

In [37]:
match_country_euro_avg_n_eu = {
    '$match':{
        'B_EU_FUNDS':'N'
    }
}

pipeline_country_euro_avg_n_eu = [match, match_value_euro, match_country_euro_avg_n_eu, group_country_euro_avg, group_country_euro_avg_2]

agg_country_euro_avg_n_eu = list(eu.aggregate(pipeline_country_euro_avg_n_eu))

agg_country_euro_avg_n_eu[0]['AVERAGE_COUNTRY_VALUE']

3428926.4692076906

11

In [38]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list}
    }
}

group_country_count = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'COUNT':{'$sum':1}
    }
}

lookup = {
    '$lookup':{
        'from':'iso_codes',
        'localField':'_id',
        'foreignField':'alpha-2',
        'as':'COUNTRY'
    }
}

project = {
    '$project':{
        '_id':False,
        'country':{ "$arrayElemAt": [ "$COUNTRY", 0] },
        'count':'$COUNT'
    }
}

project_2 = {
    '$project':{
        'country':'$country.name',
        'count':True
    }
}

pipeline = [match, group_country_count, lookup, project, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'count': 6900, 'country': 'Cyprus'},
 {'count': 59279, 'country': 'Finland'},
 {'count': 3090, 'country': 'Switzerland'},
 {'count': 30603, 'country': 'Denmark'},
 {'count': 81179, 'country': 'Latvia'},
 {'count': 87383, 'country': 'Bulgaria'},
 {'count': 34762, 'country': 'Greece'},
 {'count': 3861, 'country': 'North Macedonia'},
 {'count': 18377, 'country': 'Netherlands'},
 {'count': 88386, 'country': 'Lithuania'},
 {'count': 12628, 'country': 'Ireland'},
 {'count': 15545, 'country': 'Norway'},
 {'count': 32593, 'country': 'Slovakia'},
 {'count': 340, 'country': 'Iceland'},
 {'count': 78631, 'country': 'Slovenia'},
 {'count': 37480, 'country': 'Belgium'},
 {'count': 3488, 'country': 'Luxembourg'},
 {'count': 190353, 'country': 'Germany'},
 {'count': 21557, 'country': 'Portugal'},
 {'count': 56836, 'country': 'Hungary'},
 {'count': 140333, 'country': 'Italy'},
 {'count': 186025, 'country': 'United Kingdom'},
 {'count': 345668, 'country': 'France'},
 {'count': 2625, 'country': 'Malta

12

In [39]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'VALUE_EURO': {'$exists': True}
    }
}

group = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'AVERAGE':{'$avg':'$VALUE_EURO'}
    }
}

sort = {
    '$sort':{
        'AVERAGE': -1
    }
}

limit = {
    '$limit':5
}

lookup = {
    '$lookup':{
        'from':'iso_codes',
        'localField':'_id',
        'foreignField':'alpha-2',
        'as':'COUNTRY'
    }
}

project = {
    '$project':{
        '_id':False,
        'country':{ "$arrayElemAt": [ "$COUNTRY", 0] },
        'avg':'$AVERAGE'
    }
}

project_2 = {
    '$project':{
        'country':'$country.name',
        'avg':True
    }
}

pipeline = [match, group, sort, limit, lookup, project, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'avg': 11303641.704969117, 'country': 'Ireland'},
 {'avg': 10812171.684698293, 'country': 'United Kingdom'},
 {'avg': 7700849.07468941, 'country': 'Denmark'},
 {'avg': 6330150.5453541335, 'country': 'Norway'},
 {'avg': 5263316.019982798, 'country': 'Luxembourg'}]

13

In [40]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'VALUE_EURO': {'$exists': True}
    }
}

group = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'AVERAGE':{'$avg':'$VALUE_EURO'}
    }
}

sort = {
    '$sort':{
        'AVERAGE': 1
    }
}

limit = {
    '$limit':5
}

lookup = {
    '$lookup':{
        'from':'iso_codes',
        'localField':'_id',
        'foreignField':'alpha-2',
        'as':'COUNTRY'
    }
}

project = {
    '$project':{
        '_id':False,
        'country':{ "$arrayElemAt": [ "$COUNTRY", 0] },
        'avg':'$AVERAGE'
    }
}

project_2 = {
    '$project':{
        'country':'$country.name',
        'avg':True
    }
}

pipeline = [match, group, sort, limit, lookup, project, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'avg': 622732.6235172235, 'country': 'North Macedonia'},
 {'avg': 1123124.187401788, 'country': 'Slovenia'},
 {'avg': 1148939.245934335, 'country': 'Bulgaria'},
 {'avg': 1185984.9630768115, 'country': 'Cyprus'},
 {'avg': 1350866.0115504763, 'country': 'Malta'}]

14

In [41]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'B_EU_FUNDS': 'Y',
        'VALUE_EURO': {'$exists': True}
    }
}

group = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'SUM':{'$sum':'$VALUE_EURO'}
    }
}

lookup = {
    '$lookup':{
        'from':'iso_codes',
        'localField':'_id',
        'foreignField':'alpha-2',
        'as':'COUNTRY'
    }
}

project = {
    '$project':{
        '_id':False,
        'country':{ "$arrayElemAt": [ "$COUNTRY", 0] },
        'sum':'$SUM'
    }
}

project_2 = {
    '$project':{
        'country':'$country.name',
        'sum':True
    }
}

pipeline = [match, group, lookup, project, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'sum': 627546920.1, 'country': 'Cyprus'},
 {'sum': 2130943993.64, 'country': 'Finland'},
 {'sum': 55127375065.27, 'country': 'Latvia'},
 {'sum': 1752420246.21, 'country': 'Denmark'},
 {'sum': 8761931787.03, 'country': 'Bulgaria'},
 {'sum': 30789853883.66, 'country': 'Greece'},
 {'sum': 6367980.36, 'country': 'North Macedonia'},
 {'sum': 3119641342.61, 'country': 'Ireland'},
 {'sum': 7834030866.05, 'country': 'Lithuania'},
 {'sum': 658954626.42, 'country': 'Netherlands'},
 {'sum': 16342024427.32, 'country': 'Slovakia'},
 {'sum': 27142128.09, 'country': 'Norway'},
 {'sum': 5201424554.31, 'country': 'Slovenia'},
 {'sum': 15219058129.98, 'country': 'Belgium'},
 {'sum': 2328870042.33, 'country': 'Luxembourg'},
 {'sum': 11928777870.22, 'country': 'Germany'},
 {'sum': 7888865066.54, 'country': 'Portugal'},
 {'sum': 52934471767.6, 'country': 'Hungary'},
 {'sum': 13155538373.96, 'country': 'Italy'},
 {'sum': 155239775429.01, 'country': 'United Kingdom'},
 {'sum': 1137097615.06, 'country': 'Ma

15.1

In [42]:
#%%timeit

match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'CAE_NAME': {'$exists': True}
    }
}

match_value_euro = {
    '$match':{
        'VALUE_EURO': {'$exists': True}
    }
}

group_cae_euro_avg = {
    '$group':{
        '_id':'$CAE_NAME',
        'AVERAGE_VALUE':{'$avg':'$VALUE_EURO'}
    }
}

group_cae_euro_avg_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_CAE_VALUE':{'$avg':'$AVERAGE_VALUE'}
    }
}

pipeline_cae_euro_avg = [match, match_value_euro, group_cae_euro_avg, group_cae_euro_avg_2]

agg_business_euro_avg = list(eu.aggregate(pipeline_cae_euro_avg))

agg_business_euro_avg[0]['AVERAGE_CAE_VALUE']

2332022.339932884

15.2

In [43]:
group_cae_count = {
    '$group':{
        '_id':'$CAE_NAME',
        'COUNT':{'$sum':1}
    }
}

group_cae_count_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_COUNT':{'$avg':'$COUNT'}
    }
}

pipeline_cae_count = [match, group_cae_count, group_cae_count_2]

agg_business_count = list(eu.aggregate(pipeline_cae_count))

agg_business_count[0]['AVERAGE_COUNT']

20.86296436044318

15.3

In [44]:
match_number_offers = {
    '$match':{
        'NUMBER_OFFERS': {'$exists': True}
    }
}

group_cae_offer_avg = {
    '$group':{
        '_id':'$CAE_NAME',
        'AVERAGE_OFFERS':{'$avg':'$NUMBER_OFFERS'}
    }
}

group_cae_offer_avg_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_CAE_OFFERS':{'$avg':'$AVERAGE_OFFERS'}
    }
}

pipeline_cae_offer_avg = [match, match_number_offers, group_cae_offer_avg, group_cae_offer_avg_2]

agg_cae_offer_avg = list(eu.aggregate(pipeline_cae_offer_avg))

agg_cae_offer_avg[0]['AVERAGE_CAE_OFFERS']

5.834545217950562

15.4

In [45]:
match_cae_euro_avg_y_eu = {
    '$match':{
        'B_EU_FUNDS':'Y'
    }
}

pipeline_cae_euro_avg_y_eu = [match, match_value_euro, match_cae_euro_avg_y_eu, group_cae_euro_avg, group_cae_euro_avg_2]

agg_cae_euro_avg_y_eu = list(eu.aggregate(pipeline_cae_euro_avg_y_eu))

agg_cae_euro_avg_y_eu[0]['AVERAGE_CAE_VALUE']

2200739.469196134

15.5

In [46]:
match_cae_euro_avg_n_eu = {
    '$match':{
        'B_EU_FUNDS':'N'
    }
}

pipeline_cae_euro_avg_n_eu = [match, match_value_euro, match_cae_euro_avg_n_eu, group_cae_euro_avg, group_cae_euro_avg_2]

agg_cae_euro_avg_n_eu = list(eu.aggregate(pipeline_cae_euro_avg_n_eu))

agg_cae_euro_avg_n_eu[0]['AVERAGE_CAE_VALUE']

2321458.5092061693

16

In [47]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'CAE_NAME': {'$exists': True},
        'VALUE_EURO': {'$exists': True}
    }
}

group = {
    '$group':{
        '_id':'$CAE_NAME',
        'AVERAGE':{'$avg':'$VALUE_EURO'}
    }
}

sort = {
    '$sort':{
        'AVERAGE': -1
    }
}

limit = {
    '$limit':5
}


project = {
    '$project':{
        '_id':False,
        'company':'$_id',
        'avg':'$AVERAGE'
    }
}

pipeline = [match, group, sort, limit, project]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'company': 'Halton Housing Trust,', 'avg': 100000000.0},
 {'company': 'Rete ferroviaria italiana SpA «Gruppo Ferrovie dello Stato Italiane» — società con socio unico soggetta alla direzione e coordinamento di FS Italiane SpA, a norma dell&apos;articolo 2497 sexies cocice civile e del D.Lgs. n. 112/2015, Direzione Acquisti',
  'avg': 100000000.0},
 {'company': 'Countess of Chester Hospital NHS Foundation Trust---This Framework Agreement for use by the whole Irish public sector including the public sector organisations identified in 2015 Register of Public Sector Bodies. http://www.cso.ie/en/methods/nationalaccounts/classificationdecisions/registerofpublicsectorbodiesinireland/',
  'avg': 100000000.0},
 {'company': 'Amt der Oö. Landesregierung, Direktion Soziales u. Gesundheit, Abteilung Wohnbauförderung',
  'avg': 100000000.0},
 {'company': 'Opérateur des réseaux gaz & électricité (en abrégé ORES)',
  'avg': 100000000.0}]

17

In [48]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'CAE_NAME': {'$exists': True},
        'VALUE_EURO': {'$exists': True}
    }
}

group = {
    '$group':{
        '_id':'$CAE_NAME',
        'AVERAGE':{'$avg':'$VALUE_EURO'}
    }
}

sort = {
    '$sort':{
        'AVERAGE': 1
    }
}

limit = {
    '$limit':5
}


project = {
    '$project':{
        '_id':False,
        'company':'$_id',
        'avg':'$AVERAGE'
    }
}

pipeline = [match, group, sort, limit, project]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'company': 'Bayerische Staatsgemäldesammlungen', 'avg': 0.01},
 {'company': 'Landkreisbetriebe Neuburg-Schrobenhausen', 'avg': 0.01},
 {'company': 'Abfallwirtschaftsbetrieb Landkreis Uelzen---Landkreis Uelzen – Der Landrat',
  'avg': 0.01},
 {'company': 'Spitalfonds Meersburg, Alten- und Pflegeheim Dr. Zimmermann Stift',
  'avg': 0.01},
 {'company': 'Eigenbetrieb Abfallwirtschaft (EBA) des Rhein-Pfalz-Kreises',
  'avg': 0.01}]

18

In [49]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'CAE_NAME': {'$exists': True}
    }
}

group = {
    '$group':{
        '_id':'$CAE_NAME',
        'COUNT':{'$sum':1}
    }
}

sort = {
    '$sort':{
        'COUNT': -1
    }
}

limit = {
    '$limit':15
}


project = {
    '$project':{
        '_id':False,
        'company':'$_id',
        'count':'$COUNT'
    }
}

pipeline = [match, group, sort, limit, project]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'company': 'Uniwersyteckie Centrum Kliniczne', 'count': 12289},
 {'company': 'Szpital Kliniczny Przemienienia Pańskiego Uniwersytetu Medycznego im. Karola Marcinkowskiego w Poznaniu',
  'count': 10424},
 {'company': 'Viešoji įstaiga Centrinė projektų valdymo agentūra (126125624)',
  'count': 9794},
 {'company': 'Spitalul Clinic Judetean de Urgenta Sf. Spiridon Iasi',
  'count': 8925},
 {'company': 'Institutul Oncologic Prof. Dr. I. Chiricuta Cluj-Napoca',
  'count': 8088},
 {'company': 'Kompania Węglowa S.A.', 'count': 7915},
 {'company': 'Szpital Uniwersytecki w Krakowie', 'count': 7911},
 {'company': 'Klinički bolnički centar Rijeka', 'count': 7455},
 {'company': 'Sabiedrība ar ierobežotu atbildību “Ogres rajona slimnīca”',
  'count': 7302},
 {'company': 'Viešoji įstaiga CPO LT', 'count': 7243},
 {'company': 'Lesy Slovenskej republiky, štátny podnik', 'count': 7207},
 {'company': 'Samodzielny Publiczny Centralny Szpital Kliniczny',
  'count': 7130},
 {'company': 'Wojskowy Instytut 

19

In [63]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'CAE_NAME': {'$exists': True},
        'VALUE_EURO': {'$exists': True},
        'CAE_ADDRESS': {'$exists': True},
        'CAE_TOWN': {'$exists': True}
    }
}

group_cae_country_euro_avg = {
    '$group':{
        '_id':{'CAE_NAME':'$CAE_NAME','COUNTRY':'$ISO_COUNTRY_CODE'},
        'SUM':{'$sum':'$VALUE_EURO'},
        'ADDRESS':{'$first':'$ADDRESS'}        
    }
}

sort = {
    '$sort':{
        'SUM':-1
    }
}

group_2 = {
    '$group':{
        '_id':'$_id.COUNTRY',
        'VALUE':{'$first':'$SUM'},
        'COMPANY':{'$first':'$_id.CAE_NAME'},
        'ADDRESS':{'$first':'$ADDRESS'}
    }
}

lookup = {
    '$lookup':{
        'from':'iso_codes',
        'localField':'_id',
        'foreignField':'alpha-2',
        'as':'COUNTRY'
    }
}

project_2 = {
    '$project':{
        '_id':False,
        'country':{ "$arrayElemAt": [ "$COUNTRY", 0] },
        'company':'$COMPANY',
        'sum':'$VALUE',
        'address':'$ADDRESS'
    }
}

project_3 = {
    '$project':{
        'country':'$country.name',
        'company':True,
        'sum':True,
        'address':True
    }
}

pipeline = [match, group_cae_country_euro_avg, sort, group_2, lookup, project_2, project_3]

list_documents = list(eu.aggregate(pipeline,allowDiskUse=True))

list_documents

[{'company': 'European Commission',
  'sum': 13882139733.05,
  'address': "avenue d'Auderghem 45 Brussels",
  'country': 'Belgium'},
 {'company': 'Publications Office of the European Union',
  'sum': 8826706789.52,
  'address': '2, rue Mercier Luxembourg',
  'country': 'Luxembourg'},
 {'company': 'Landschaftsverband Rheinland',
  'sum': 5774534049.22,
  'address': 'Ottoplatz 2 Köln',
  'country': 'Germany'},
 {'company': 'Parque Escolar E.P.E.',
  'sum': 2294873572.34,
  'address': 'Avenida Infante Santo nº2, 7º andar Lisboa',
  'country': 'Portugal'},
 {'company': 'Közbeszerzési és Ellátási Főigazgatóság',
  'sum': 18563289618.19,
  'address': 'Margit krt. 85-87. Budapest',
  'country': 'Hungary'},
 {'company': 'Scotland Excel',
  'sum': 124784964570.25,
  'address': 'Unit 5, Anchor Mill, 7 Thread Street Paisley',
  'country': 'United Kingdom'},
 {'company': 'Società Regionale per la Sanità (SO.RE.SA. SpA)',
  'sum': 33850655961.43,
  'address': 'Centro Direzionale Isola C 1 Torre Sav

20

In [68]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'CAE_NAME': {'$exists': True},
        'WIN_NAME': {'$exists': True},
    }
}

group = {
    '$group':{
        '_id':'$PAIR',
        'COUNT':{'$sum':1}
    }
}

sort = {
    '$sort':{
        'COUNT':-1
    }
}

limit = {
    '$limit':5
}

project_2 = {
    '$project':{
        '_id':False,
        'companies':'$_id',
        'count':'$COUNT'
    }
}

pipeline = [match, group, sort, limit, project_2]

list_documents = list(eu.aggregate(pipeline,allowDiskUse=True))

list_documents

[{'companies': 'Viešoji įstaiga Centrinė projektų valdymo agentūra (126125624) with UAB "Limedika" (134056779)',
  'count': 1741},
 {'companies': "Viešoji įstaiga Centrinė projektų valdymo agentūra (126125624) with UAB 'Limedika' (134056779)",
  'count': 1626},
 {'companies': 'Sabiedrība ar ierobežotu atbildību “Ogres rajona slimnīca” with SIA “Magnum Medical”',
  'count': 1441},
 {'companies': 'Viešoji įstaiga Centrinė projektų valdymo agentūra (126125624) with UAB "ARMILA" (123813957)',
  'count': 1220},
 {'companies': "Viešoji įstaiga „CPO LT“ with UAB 'Limedika'", 'count': 1164}]

In [41]:
from bson.code import Code
mapper = Code(
    """
       function () {
            emit((this.CAE_NAME,this.WIN_NAME), 1);
       }
   """
)

reducer = Code(
    """
        function (key, values) {
          var total = 0;
          for (var i = 0; i < values.length; i++) {
            total += values[i];
          }
          return total;
        }
    """
)

eu.map_reduce(mapper, reducer, "myresults")

AutoReconnect: connection closed